In [1]:
import numpy as np
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


In [2]:
molecule = vlx.Molecule.read_str("Ne  0.000 0.000 0.000")

In [3]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

lrf_drv = vlx.LinearResponseSolver()
lrf_drv.ostream.mute()

lrf_drv.b_operator = "electric dipole"

lrf_drv.a_components = ["z"]
lrf_drv.b_components = ["z"]

w = 0.0656
lrf_drv.frequencies = [w]

In [16]:
print("-" * 44)
print(f"{'Basis set':12s}{'-<<z;z>>':>16s}{'-i/w*<<p_z;z>>':>16s}")
print("-" * 44)

for basis_set in ["aug-cc-pVDZ", "daug-cc-pVDZ"]:
    basis = vlx.MolecularBasis.read(molecule, basis_set, ostream=None)

    scf_results = scf_drv.compute(molecule, basis)

    lrf_drv.a_operator = "electric dipole"
    lrf_results = lrf_drv.compute(molecule, basis, scf_results)
    lrf_length = lrf_results["response_functions"][("z", "z", w)]

    lrf_drv.a_operator = "linear momentum"
    lrf_results = lrf_drv.compute(molecule, basis, scf_results)
    lrf_velocity = lrf_results["response_functions"][("z", "z", w)]

    print(f"{basis_set :>12s}{-lrf_length :16.6f}{lrf_velocity / w :16.6f}")

print("-" * 44)

--------------------------------------------
Basis set           -<<z;z>>  -i/w*<<p_z;z>>
--------------------------------------------
 aug-cc-pVDZ        1.837316        1.720512
daug-cc-pVDZ        2.362190        2.377629
--------------------------------------------
